In [ ]:
# | default_exp index_tracking


In [ ]:

# | export
from sqlmodel import Session, select
from seo_rat.models import IndexStatus
from seo_rat.gsc_client import GSCAuth
from googleapiclient.discovery import build
from typing import List, Optional


# | export
def inspect_url_status(auth: GSCAuth, site_url: str, page_url: str) -> dict:
    """Inspect URL indexing status from GSC"""
    service = build("searchconsole", "v1", credentials=auth.get_credentials())

    request = {"inspectionUrl": page_url, "siteUrl": site_url, "languageCode": "en-US"}

    response = service.urlInspection().index().inspect(body=request).execute()
    result = response.get("inspectionResult", {}).get("indexStatusResult", {})

    return {
        "verdict": result.get("verdict", "UNKNOWN"),
        "coverage_state": result.get("coverageState"),
        "last_crawl_time": result.get("lastCrawlTime"),
        "indexing_state": result.get("indexingState"),
        "robots_txt_state": result.get("robotsTxtState"),
    }


In [ ]:
# | export
def store_index_status(session: Session, auth: GSCAuth, site_url: str, page_url: str):
    """Inspect and store URL index status"""
    status_data = inspect_url_status(auth, site_url, page_url)

    # Upsert
    existing = session.exec(
        select(IndexStatus).where(IndexStatus.page_url == page_url)
    ).first()

    if existing:
        for key, value in status_data.items():
            setattr(existing, key, value)
        existing.checked_at = datetime.now()
    else:
        index_status = IndexStatus(site_url=site_url, page_url=page_url, **status_data)
        session.add(index_status)

    session.commit()



In [ ]:

# | export
def get_index_status(
    session: Session, site_url: str, verdict: str = None
) -> List[IndexStatus]:
    """Get stored index status for a site"""
    stmt = select(IndexStatus).where(IndexStatus.site_url == site_url)
    if verdict:
        stmt = stmt.where(IndexStatus.verdict == verdict)
    return session.exec(stmt).all()
